In [0]:
import torch
import torch.nn as nn
# 신경망 모델들이 포함되어있다.
import torch.optim as optim
#경사하강법 알고리즘이 포함되어있다.
import torch.nn.init as init
#텐서에 초기값을 주기위해 필요한 함수들이있다.
import matplotlib.pyplot as plt
import torchvision.datasets as dset
import torchvision.transforms as transforms
# torchvision은 유명한 영상처리용 데이터 셋, 모델, 이미지 변환기가 들어있는 패키지
# dataset 모듈은 데이터를 읽어오는 역할, transforms는 불러온 이미지를 필요에 따라 변환해주는 역할
from torch.utils.data import DataLoader
# 데이터를 하나씩 전달하지않고 원하는 배치사이즈대로 묶어서 전달하거나 더 효율적인 학습을 위해 데이터를
# 어떤 규칙에 따라 정렬하거나 섞어줄때 해주는 역할
 
 
 
batch_size= 1
learning_rate = 0.0002
num_epoch = 100

In [0]:
def conv_1(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,1,1),
        nn.ReLU(),
    )
    return model

def conv_1_3(in_dim,mid_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,mid_dim,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_dim,out_dim,3,1,1),
        nn.ReLU()
    )
    return model
    
def conv_1_5(in_dim,mid_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,mid_dim,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_dim,out_dim,5,1,2),
        nn.ReLU()
    )
    return model
    

def max_3_1(in_dim,out_dim):
    model = nn.Sequential(
        nn.MaxPool2d(3,1,1),
        # 커널 사이즈 , 스트라이드, 패딩 , maxpooling 은 최대값만 남겨준다. 
        nn.Conv2d(in_dim,out_dim,1,1),
        nn.ReLU(),
    )
    return model

In [0]:
class inception_module(nn.Module):
  # 원본 filter=1 , filter =3 , filter 5 인 conv , 
  def __init__(self,in_dim,out_dim_1,mid_dim_3,out_dim_3,mid_dim_5,out_dim_5,pool):
    super(inception_module,self).__init__()
    self.conv_1 = conv_1(in_dim,out_dim_1)
    self.conv_1_3 = conv_1_3(in_dim,mid_dim_3,out_dim_3)
    self.conv_1_5 = conv_1_5(in_dim,mid_dim_5,out_dim_5)
    self.max_3_1 = max_3_1(in_dim,pool)


  def forward(self,x):
    out_1 = self.conv_1(x)
    out_2 = self.conv_1_3(x)
    out_3 = self.conv_1_5(x)
    out_4 = self.max_3_1(x)
    output = torch.cat([out_1,out_2,out_3,out_4],1)
    # 하나로 다 합쳐줌 
    return output

In [0]:
# 표와 비교하면서 볼 수 있도록 구현했습니다. 

class GoogLeNet(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(GoogLeNet, self).__init__()
        self.num_classes=num_classes
        self.layer_1 = nn.Sequential(
            # base_dim = 64
            nn.Conv2d(3,base_dim,7,2,3),
            nn.MaxPool2d(3,2,1),
            nn.Conv2d(base_dim,base_dim*3,3,1,1),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_2 = nn.Sequential(
            inception_module(base_dim*3,64,96,128,16,32,32),
            inception_module(base_dim*4,128,128,192,32,96,64),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_3 = nn.Sequential(
            inception_module(480,192,96,208,16,48,64),
            inception_module(512,160,112,224,24,64,64),
            inception_module(512,128,128,256,24,64,64),
            inception_module(512,112,144,288,32,64,64),
            inception_module(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_4 = nn.Sequential(
            inception_module(832,256,160,320,32,128,128),
            inception_module(832,384,192,384,48,128,128), 
            nn.AvgPool2d(7,1),
        )
        self.layer_5 = nn.Dropout2d(0.4)
        self.fc_layer = nn.Linear(1024,self.num_classes)
                
        
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out
      

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = GoogLeNet(base_dim=64)
for i in model.named_children():
    print(i)

model = GoogLeNet(base_dim=64).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

cpu
('layer_1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
))
('layer_2', Sequential(
  (0): inception_module(
    (conv_1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
    )
    (conv_1_3): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
    )
    (conv_1_5): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (3): ReLU()
    )
    (max_3_1): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, paddin

In [0]:
def conv_block_1(in_dim,out_dim,act_fn,stride=1):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=1,stride=stride),
      act_fn,
  )
  return model

def conv_block_3(in_dim,out_dim,act_fn):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,stride=1,padding = 1),
      act_fn,
  )
  return model


In [0]:
class BottleNeck(nn.Module):
    def __init__(self,in_dim,mid_dim,out_dim,act_fn,down=False):
        super(BottleNeck,self).__init__()
        self.down=down
        # 블록을 통과했을 때 특성 지도의 크기가 줄어드는지 여부를 물어보는 Boolean 
        # 줄어들 경우 스트라이드가 2가되어 크기가 반으로 줄어든다
        
        # 특성지도의 크기가 감소하는 경우
        if self.down:
            self.layer = nn.Sequential(
              conv_block_1(in_dim,mid_dim,act_fn,2),
              conv_block_3(mid_dim,mid_dim,act_fn),
              conv_block_1(mid_dim,out_dim,act_fn),
            )
            self.downsample = nn.Conv2d(in_dim,out_dim,1,2)
            
        # 특성지도의 크기가 그대로인 경우
        else:
            self.layer = nn.Sequential(
                conv_block_1(in_dim,mid_dim,act_fn),
                conv_block_3(mid_dim,mid_dim,act_fn),
                conv_block_1(mid_dim,out_dim,act_fn),
            )
            
        # 더하기를 위해 차원을 맞춰주는 부분
        self.dim_equalizer = nn.Conv2d(in_dim,out_dim,kernel_size=1)
                  
    def forward(self,x):
        if self.down:
            downsample = self.downsample(x)
            out = self.layer(x)
            out = out + downsample
        else:
            out = self.layer(x)
            if x.size() is not out.size():
                x = self.dim_equalizer(x)
                # 채널의 개수가 달라서 더해지지 않은 경우에 차원 맞춰주기위해서 1x1 합성곱을 추가해 입력, 출력 채널 같게 만들어준다
            out = out + x
        return out


In [0]:
class ResNet(nn.Module):

  def __init__(self,base_dim,num_classes=2):
    super(ResNet,self).__init__()
    self.act_fn = nn.ReLU()
    self.layer_1 = nn.Sequential(
        nn.Conv2d(3,base_dim,7,2,3),
        nn.ReLU(),
        nn.MaxPool2d(3,2,1),
    )
    self.layer_2 = nn.Sequential(
        BottleNeck(base_dim,base_dim,base_dim*4,self.act_fn),
        BottleNeck(base_dim*4,base_dim,base_dim*4,self.act_fn),
        BottleNeck(base_dim*4,base_dim,base_dim*4,self.act_fn,down=True),
    )
    self.layer_3 = nn.Sequential(
        BottleNeck(base_dim*4,base_dim*2,base_dim*8,self.act_fn),
        BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn),
        BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn),
        BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn,down=True),
    )
    self.layer_4 = nn.Sequential(
        BottleNeck(base_dim*8,base_dim*4,base_dim*16,self.act_fn),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn,down=True),
    )
    self.layer_5 = nn.Sequential(
        BottleNeck(base_dim*16,base_dim*8,base_dim*32,self.act_fn),
        BottleNeck(base_dim*16,base_dim*8,base_dim*32,self.act_fn),
        BottleNeck(base_dim*16,base_dim*8,base_dim*32,self.act_fn),
    )
    self.avgpool = nn.AvgPool2d(7,1)
    self.fc_layer = nn.Linear(base_dim*32,num_classes)

    def forward(self,x):
      out = self.layer_1(x)
      out = self.layer_2(out)
      out = self.layer_3(out)
      out = self.layer_4(out)
      out = self.layer_5(out)
      out = self.avgpool(out)
      out = out.view(batch_size,-1)
      out = self.fc_layer(out)
      return out
